## Project 4
In this task we will perform sequence classification. We will categorize temporally coherent and uniformly distributed short sections of a long time-series. In particular, for each 4 seconds of a lengthy EEG/EMG measurement of brain activity recorded during sleep, we will assign one of the 3 classes corresponding to the sleep stage present within the evaluated epoch.

Each row in train_{eeg1,eeg2,emg}.csv is a single epoch of the corresponding channel indexed by an id, so the first column contains the id. In addition to the id column, each sample has 512 values corresponding to 4x128, where 4 is the number of seconds per epoch, and 128 is the measurement frequency. Note also that the data contains stacked recordings of three subjects. Each subject has 21600 epochs (24 hours) rendering in total 3x21600=64800 epochs i.e. training data points. Therefore, apart from the "breaks" between subjects, neighboring epochs are temporally coherent. The file structure is therefore:

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import correlate
from scipy.fft import fft
from statsmodels.tsa.stattools import adfuller

In [2]:
train_eeg1 = pd.read_csv('raw/train_eeg1.csv', index_col='Id')
train_eeg2 = pd.read_csv('raw/train_eeg2.csv', index_col='Id')
train_emg = pd.read_csv('raw/train_emg.csv', index_col='Id')
train_labels = pd.read_csv('raw/train_labels.csv', index_col='Id')

print('Shape of data')
print('Shape of train_eeg1 ', train_eeg1.shape)
print('Shape of train_eeg2 ', train_eeg2.shape)
print('Shape of train_emg ', train_emg.shape)
print()
print('Head of eeg1')
train_eeg1.head()


Shape of data
Shape of train_eeg1  (64800, 512)
Shape of train_eeg2  (64800, 512)
Shape of train_emg  (64800, 512)

Head of eeg1


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x503,x504,x505,x506,x507,x508,x509,x510,x511,x512
Id,,,,,,,,,,,,,,,,,,,,,
0,0.000400,0.000470,0.000067,-0.00016,-0.000003,0.00031,0.000360,0.00019,-0.000072,-0.00007,...,-0.000086,0.000033,-0.000046,-0.000270,-0.000390,-0.00034,-0.00032,-0.00021,0.000042,0.000053
1,0.000067,0.000095,0.000270,0.00028,0.000250,0.00012,0.000094,-0.00034,-0.000960,-0.00120,...,0.000046,0.000300,0.000630,0.000710,0.000520,0.00041,0.00066,0.00088,0.000770,0.000410
2,0.000160,-0.000210,-0.000840,-0.00120,-0.001200,-0.00140,-0.001400,-0.00091,-0.000600,-0.00027,...,-0.000680,-0.000880,-0.001000,-0.000770,-0.000680,-0.00073,-0.00073,-0.00062,-0.000550,-0.000540
3,-0.000140,0.000260,0.000390,0.00043,0.000280,0.00023,0.000390,0.00022,0.000150,0.00022,...,0.000720,0.000760,0.000380,0.000052,-0.000260,-0.00058,-0.00075,-0.00110,-0.001200,-0.001200
4,-0.001100,-0.000790,-0.000081,0.00014,0.000200,-0.00014,-0.000430,-0.00053,-0.000580,-0.00041,...,0.000290,0.000600,0.000670,0.000190,-0.000055,-0.00016,-0.00023,-0.00023,-0.000330,-0.000810


In [3]:
train_emg.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x503,x504,x505,x506,x507,x508,x509,x510,x511,x512
Id,,,,,,,,,,,,,,,,,,,,,
0,-0.000024,-0.000018,-0.000014,-0.000017,-0.000012,-0.000007,-0.000004,-0.000009,-0.000060,-0.000031,...,-0.000003,-0.000009,-0.000010,-0.000002,-9.000000e-07,-0.000010,-0.000006,-0.000013,-0.000030,-0.000004
1,-0.000005,-0.000007,-0.000007,-0.000008,-0.000006,-0.000008,-0.000012,-0.000017,-0.000015,-0.000010,...,-0.000014,-0.000016,-0.000010,-0.000005,-9.200000e-06,-0.000011,-0.000010,-0.000015,0.000006,-0.000006
2,-0.000013,-0.000013,-0.000008,-0.000014,-0.000005,-0.000008,-0.000011,-0.000007,-0.000008,-0.000008,...,-0.000005,-0.000008,-0.000011,-0.000014,-1.800000e-05,-0.000012,-0.000015,-0.000015,-0.000016,-0.000011
3,-0.000011,-0.000010,-0.000029,-0.000041,-0.000010,-0.000021,-0.000010,-0.000012,-0.000005,-0.000007,...,0.000019,-0.000012,-0.000009,-0.000011,-6.400000e-06,-0.000012,-0.000008,-0.000007,-0.000003,-0.000014
4,-0.000011,-0.000014,-0.000010,-0.000011,-0.000004,-0.000051,-0.000029,-0.000002,-0.000011,-0.000013,...,-0.000033,-0.000005,-0.000008,-0.000011,-1.200000e-05,-0.000008,-0.000008,-0.000007,-0.000011,-0.000007


In [4]:
#missing data
print(train_eeg1.isna().values.any())
print(train_eeg2.isna().values.any())
print(train_emg.isna().values.any())

#total = train_eeg1.isnull().sum().sort_values(ascending=False)
#percent = (train_eeg1.isnull().sum()/train_eeg1.isnull().count()).sort_values(ascending=False)
#missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
#missing_data.head(20)

False
False
False


In [34]:
###Install this and restart from terminal
#conda install -y nodejs
#pip install --upgrade jupyterlab
#jupyter labextension install @jupyter-widgets/jupyterlab-manager
#jupyter labextension install jupyter-matplotlib
#jupyter nbextension enable --py widgetsnbextension

%matplotlib widget

train_eeg1.iloc[1].plot()
train_eeg2.iloc[1].plot()
train_emg.iloc[1].plot()
plt.legend(['eeg1', 'eeg2', 'emg'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
#Function to check the stationarity of the time serie using Dickey fuller test
def stationarity(ts):
    print('Results of Dickey-Fuller Test:')
    test = adfuller(ts, autolag='AIC')
    results = pd.Series(test[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for i,val in test[4].items():
        results['Critical Value (%s)'%i] = val
    print (results)

stationarity(train_eeg1.iloc[1])
#https://machinelearningmastery.com/time-series-data-stationary-python/

Results of Dickey-Fuller Test:
Test Statistic                -8.793585e+00
p-value                        2.206874e-14
#Lags Used                     5.000000e+00
Number of Observations Used    5.060000e+02
Critical Value (1%)           -3.443340e+00
Critical Value (5%)           -2.867269e+00
Critical Value (10%)          -2.569821e+00
dtype: float64


In [31]:
def autocorr(x):
    return np.correlate(x, x, mode='same')
%matplotlib widget

#Data looks kind of periodic, try autcorrelation
plt.plot(autocorr(train_eeg1.iloc[1]))
plt.title('Autocorrelation EEG Canal1')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

How about cross correlation

In [29]:
%matplotlib widget
plt.plot(correlate(train_eeg2.iloc[1], train_eeg1.iloc[1]))
plt.title('Cross correlation EEG Canals')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
%matplotlib widget
eeg1f = fft(train_eeg1.iloc[1].values)
N = train_eeg1.shape[1]
T = 4/N
print('Sampling Frequency', 1/T, 'Hz')
xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
plt.plot(xf, 2.0/N * np.abs(eeg1f[0:N//2]))
plt.grid()
plt.show()

Sampling Frequency 128.0 Hz


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We can lowpass this bitch with cutoff at about 50Hz to get smoother signal